In [ ]:
using Plots; gr()
using Reproduce
using FileIO
using Statistics
using ProgressMeter
using JLD2
# data_home = "../data/experiment_2/Experiment2"
# data_home = "../Experiment2_ESarsa_NoExploration"
# data_home = "../Experiment2_NoExploringStarts"
# data_home = "../Experiment2_Non_Introspective"
# data_home = "../data/Experiment2_Introspective_And_Nonintrospective"
# data_home = "../data/Experiment2_Introspective"
# data_home = "../data/Revised_Experiment2"
# data_home = "../data/OneDTMaze_Revised_RoundRobin"
# data_home = "../OneDTMaze_ESARSA_Control_No_Penalty"
# data_home = "../data/OneDTMaze_GPI_Control_Part_1"
# data_home = "../data/OneDTMaze_Control_Part_1_Maze_Rep"
data_home = "../OneDTMaze_GPI_Leave"

In [ ]:
include("./plot_utils.jl")
GPU = GeneralPlotUtils

In [ ]:
# experiment_folders = ["../data/Experiment1 (copy)"]
# experiment_folders = ["../Experiment1"]
# experiment_folders = ["../data/Experiment1Final"]
experiment_folders = [data_home]
folder_name = "onedtmaze_simplified"

In [ ]:
data_key = :oned_tmaze_start_error

In [ ]:
ic = ItemCollection(joinpath(experiment_folders[1], "data"));
# ic = search(ic, Dict("env_step_penalty" => -0.01))
diff(ic)

In [ ]:
# length(search(ic, Dict("demon_learner" => "LSTD")))
# ic = search(ic, Dict("num_tilings"=>2))

In [ ]:
# algo_divisor_keys = ["behaviour_learner","behaviour_update","demon_learner", "demon_opt", "demon_update", "demon_alpha_init"]
# algo_divisor_keys = ["behaviour_learner", "behaviour_update", "demon_learner", "demon_opt"]
# algo_divisor_keys = ["behaviour_learner","behaviour_reward_projector", "demon_learner","num_tiles", "exploration_param", "exploring_starts"]
# algo_divisor_keys = ["behaviour_learner","demon_learner", "demon_opt"]
# algo_divisor_keys = ["demon_learner", "demon_update", "demon_opt","demon_rep"]
algo_divisor_keys = ["behaviour_w_init"]
# algo_divisor_keys = ["demon_learner"]
# algo_divisor_keys = ["behaviour_eta"]
# algo_divisor_keys = []
# sweep_params = ["demon_eta"]
# sweep_params = ["demon_eta", "behaviour_eta", "exploration_param"]
sweep_params = []

algo_specs_full = GPU.split_algo(ic, algo_divisor_keys)
# algo_specs_full = ic

In [ ]:
all_algos_ics = [search(ic,algo_spec) for algo_spec in algo_specs_full]
# all_algos_ics = [search(ic,algo_specs_full)]
length.(all_algos_ics)


In [ ]:
valid_algos_ind = findall(a -> length(a) != 0, all_algos_ics)

algo_specs = algo_specs_full[valid_algos_ind]
algo_ics = all_algos_ics[valid_algos_ind]

In [ ]:
algo_specs

In [ ]:
algo_ics[1].items

In [ ]:
best_per_algo_ics = []
include("./plot_utils.jl")
GPU = GeneralPlotUtils
for (i,algo_ic) in enumerate(algo_ics)
    println(i)
    push!(best_per_algo_ics, GPU.get_best(algo_ic,sweep_params, data_key))
#     push!(best_per_algo_ics, GPU.get_best_final_perf(algo_ic,sweep_params, data_key, 0.1))

end

In [ ]:
length.(best_per_algo_ics)

In [ ]:
data = [GPU.smooth(GPU.get_stats(GPU.load_results(ic,data_key))[1],10) for ic in best_per_algo_ics]
std = [GPU.smooth(GPU.get_stats(GPU.load_results(ic,data_key))[2],10) for ic in best_per_algo_ics]

size(GPU.load_results(best_per_algo_ics[1], data_key))
# GVF x timesteps x num_runs
# tst = GPU.load_results(best_per_algo_ics[1], data_key)
# size(tst)
# std(sum(tst,dims=1)[1,:,:], dims =2)
# methods(std)
# std(tst,dims=3)
# GPU.get_stats(GPU.load_results(best_per_algo_ics[1],data_key))[2]

In [ ]:
algo_specs
[GPU.print_params(algo, algo_divisor_keys, sweep_params) for algo in best_per_algo_ics]

In [ ]:
label_keys = cat(algo_divisor_keys,sweep_params, dims = 1)
labels = [GPU.get_label(algo, label_keys) for algo in best_per_algo_ics]
labels = cat(labels..., dims=2)

In [ ]:
# labels = [GPU.get_label(algo, label_keys) for algo in best_per_algo_ics]
ylabel = "RMSE"
# title = "SR Demons & Step Size Adaptation vs More Naive Approaches"
title = "ESARSA: Criteria - Last 10%"
step_increment=100
num_samples = length(data[1])
xticks=collect(step_increment:step_increment:num_samples*step_increment)
num_runs = 15
plot(xticks, data, ylabel=ylabel, palette=:tab10, label= labels, grid=true, ribbon = std/sqrt(num_runs), legend=:topright, title=title)

In [ ]:
savefig("./plots/$(folder_name)/RMSE.png")

In [ ]:
data_per_gvf = [GPU.get_stats(GPU.load_results(ic,data_key), per_gvf = true)[1] for ic in best_per_algo_ics]
std_per_gvf = [GPU.get_stats(GPU.load_results(ic,data_key), per_gvf = true)[2] for ic in best_per_algo_ics]

In [ ]:
std_per_gvf[1][1,:]

In [ ]:
num_gvfs = 4
ps = []
gvf_labels = ["Distractor" "Constant" "Drifter" "Constant"]
for gvf_ind in 1:num_gvfs
    p = plot()
    for algo_ind in 1:length(data_per_gvf)
        smooth_gvf = GPU.smooth(data_per_gvf[algo_ind][gvf_ind,:],5)
        plot!(p,smooth_gvf, palette=:tab10, ribbon = std_per_gvf[algo_ind][gvf_ind,:] / sqrt(15), size = (500,500),label = labels[algo_ind])
        plot!(xlabel="Steps", ylabel = "RMSE",  title = gvf_labels[gvf_ind])
    end
    push!(ps,p)
end
plot(ps..., layout=(2,2), size = (1200,1200))
        

In [ ]:
savefig("./plots/$(folder_name)/RMSE_per_Demon.png")

In [ ]:
epag = [GPU.load_results(ic,data_key) for ic in best_per_algo_ics]
epag[1]
for (algo_ind,tmp) in enumerate(epag)
    num_runs = size(tmp)[3]
    ps = []
    for gvf_ind in 1:num_gvfs
        p = plot()
        for i in 1:num_runs
            plot!(tmp[gvf_ind,:,i],linealpha=0.3,label="",color=:black)
        end
        plot!(data_per_gvf[algo_ind][gvf_ind,:], label = labels[algo_ind])
        title!(gvf_labels[gvf_ind])
        push!(ps,p)
    end
    plot(ps...)
#     savefig("./plots/onedtmaze_roundrobin/rmse_per_run_$(labels[algo_ind]).png")
end

    

In [ ]:
ps = []
inds_to_care = [1:4,5:8,9:13,14:18]
for inds in inds_to_care
    ylabel = "RMSE"
    title = ""
    step_increment=100
    num_samples = length(data[1])
    xticks=collect(step_increment:step_increment:num_samples*step_increment)
    num_runs = 50
    ribbon_std_err = std[inds] / sqrt(num_runs)
    p = plot(xticks, data[inds], ylabel=ylabel, palette=:tab10, label= labels[:,inds], grid=true, ribbon = ribbon_std_err, legend=:topright, title=title)
    push!(ps,p)
end
plot(ps..., size = (800,800))

In [ ]:
savefig("./plots/experiment_2_revised/introspective_metastepsize_longer.png")

In [ ]:
#Replicating plot from Experiment 1.
# inds_to_replicate = [1,3,4,8]
inds_to_replicate = [6,8]
label_subset = labels[:,inds_to_replicate]
data_subset = data[inds_to_replicate]
@show length(std)
label_subset

In [ ]:
# title = "High vs Low Alpha Init"
title = "SR Learners"
ylabel = "RMSE"
step_increment=100
num_samples = length(data[1])
xticks=collect(step_increment:step_increment:num_samples*step_increment)
plot(xticks, data_subset,ylabel= ylabel, label= label_subset, ribbon = std[inds_to_replicate]/sqrt(30), legend=:topright, title=title, ylims=(0,8))

In [ ]:
savefig("./plots/onedtmaze_roundrobin/SR_learners.png")

In [ ]:
for e in diff(ic)["exploration_param"]
    @show e
end

In [ ]:
inds_to_replicate = [5,6,7]
label_subset = labels[:,inds_to_replicate]
data_subset = data[inds_to_replicate]

In [ ]:
title = "Behaviour Learned vs Round Robin"
ylabel = "RMSE"
step_increment=100
num_samples = length(data[1])
xticks=collect(step_increment:step_increment:num_samples*step_increment)
p1 = plot(xticks, data_subset,ylabel= ylabel, label= label_subset, ribbon = std[inds_to_replicate]/sqrt(30), legend=:topright, title=title, ylims=(0,4))

start_pos = 150
new_x_ticks = collect(step_increment * start_pos:step_increment:num_samples*step_increment)
data_subset_cut = [s[start_pos:end] for s in data_subset]
std_subset_cut = [s[start_pos:end] for s in std[inds_to_replicate]/sqrt(30)]
p2 = plot(new_x_ticks,data_subset_cut, ribbon = std_subset_cut, title="Cut at $(step_increment * start_pos)")

plot(p1,p2, layout = (2,1), figsize=(20,20))


In [ ]:
savefig("./plots/experiment2/learned_vs_round_robin.png")

In [ ]:
episode_lengths = [GPU.load_results(ic,:episode_length, return_type = "array") for ic in best_per_algo_ics]
visit_orders = [GPU.load_results(ic,:visit_order, return_type = "array") for ic in best_per_algo_ics]

In [ ]:
length(visit_orders[1][7])
# length(episode_lengths[1][7])

In [ ]:
ps = []
for ind in 1:length(episode_lengths)
    num_episodes = [length(e) for e in episode_lengths[ind]]
#     bins = (1:10,11:100,101:300,301:1000,1001:10000)
#     bins = 1:10:100
#     @show sum(num_episodes), labels[ind]
#     p = histogram(num_episodes; bins)
    p = histogram(num_episodes; nbins=15, xlim=(0,3000))
#     p = histogram(num_episodes; nbins=15)

    title!(labels[ind])
    push!(ps,p)
    xlabel!("Number of Episodes")
    ylabel!("Number of runs")
end

plot(ps..., size = (1000,1000))

In [ ]:
savefig("./plots/$(folder_name)/RunSuccessRate_ALL.png")

In [ ]:
data_per_run = [GPU.load_results(ic,data_key) for ic in best_per_algo_ics]
data_per_run[1]

In [ ]:
for ind in 1:length(data_per_run)
    ps = []
    d = data_per_run[ind]
    for gvf_i in 1:4
        p = plot(d[gvf_i,:,:], linecolor=:blue, linealpha=0.3, label = "")
        plot!(mean(d[gvf_i,:,:], dims = 2), linewidth=5, label = "")
        title!(string(gvf_labels[gvf_i], " ", labels[ind]))
        push!(ps,p)
    end
    full_p = plot(ps..., size = (800,800))
    display(full_p)
#     savefig("./plots/onedtmaze_revised/RMSE_per_run_$(labels[ind])_esarsa.png")
end


In [ ]:
maximum([e[end-1] for e in episode_lengths[2]])

In [ ]:
max_lengths = [GPU.get_min_length(arrs)-1 for arrs in episode_lengths]


In [ ]:
p = plot(xlabel = "Episode Count", ylabel = "Step Length")
for (ind,episode_lengths) in enumerate(episode_lengths)
    l = hcat([epi[1:max_lengths[ind]] for epi in episode_lengths]...)
    plot!(p,mean(l,dims=2), label = labels[ind],yaxis=:log)
end
title!("Step Length for Best Performing Algos Last 10 %")
# p = plot(mean(episode_lengths,dims=2), xlabel="Episode Count", ylabel="Step Length", hline=5.5)
display(p)
# savefig("./plots/no_reward/step_length.png")

In [ ]:
ps = []
for (algo_ind,algo) in enumerate(episode_lengths)
    p = plot()
    num_runs = length(algo)
    num_steps = cumsum(algo[1])[end]
    d = zeros(num_runs, num_steps)
    for (i,run) in enumerate(algo)
        cur_index = 1
        for (ep_num,ep) in enumerate(run)
            d[i,cur_index:cur_index + ep-1] .= ep
            cur_index += ep
        end            
#             @show d[i,:]
            plot!(d[i,:],color=:black, linealpha=0.4, label="")
    end
    plot!(mean(d,dims=1)', label = labels[algo_ind], yaxis=:log, size=(1000,1000), legend=:bottomleft)
    xlabel!("Step")
    ylabel!("Ave Ep Length at Step")
    push!(ps, p)
    
end

plot(ps...)
        

In [ ]:
savefig("./plots/$(folder_name)/ep_length_per_step.png")

In [ ]:
# include("./plot_utils.jl")
# GPU = GeneralPlotUtils
gvf_labels = ["Distractor" "Constant" "Drifter" "Constant"]

In [ ]:
ps = []
for i in 1:length(visit_orders)
    @show max_lengths[i]
    @show length(visit_orders[i])
    visit_perc = GPU.goal_visits_per_episode(visit_orders[i], max_lengths[i])
    @show size(visit_perc)
#     @show size(GPU.smooth(visit_perc, 5))
#     if size(visit_perc)[2] != 0
#         smooth_visit_perc = GPU.smooth(visit_perc, 5)
#     end
    tmp = [GPU.smooth(visit_perc[i,:],20) for i in 1:4] 
    p = plot(tmp, labels = gvf_labels, xlabel="Episode Count", ylabel="Fraction of Goal Visits", ylim=(0.0,1.0), title = string(labels[i], " Last 10%"), xaxis=:log, size = (800,1200), legend=:topleft)
    push!(ps,p)
end
println()
@show length(visit_orders)
# plot(ps..., layout = (1,3))
# display(plot(ps...,layout = (2,1)))
# display(plot(ps...))
display(plot(ps..., layout=(1,2)))
savefig("./plots/$(folder_name)/goal_visits.png")

In [ ]:
tmp_lengths = [length.(visits) for visits in visit_orders]
med_visit_order_lengths = Int.(floor.([median(l) for l in tmp_lengths]))
med_visit_mask = [tmp_lengths[i] .> med_visit_order_lengths[i] for i in 1:length(visit_orders)]
med_visit_order_lengths

In [ ]:

ps = []
for i in 1:length(visit_orders)
    visit_perc = GPU.goal_visits_per_episode(visit_orders[i][med_visit_mask[i]], med_visit_order_lengths[i])
    tmp = [GPU.smooth(visit_perc[i,:],100) for i in 1:4] 
    p = plot(tmp, labels = gvf_labels, xlabel="Episode Count", ylabel="Fraction of Goal Visits", ylim=(0.0,1.0), title = string("MEDIAN",labels[i], " Last 10%"), size = (800,1200), legend=:topleft)
    push!(ps,p)
end
println()
@show length(visit_orders)
display(plot(ps..., layout=(3,2)))
savefig("./plots/$(folder_name)/goal_visits_MEDIAN.png")

In [ ]:
tmpind = 13
for i in 1:length(visit_orders[tmpind])
    println(i)
    histogram(visit_orders[tmpind][i], title=string(labels[tmpind], " seed $(i)", " Last visit: $(visit_orders[tmpind][i][end])"), yaxis=:log)
    savefig("./plots/no_reward/goal_visitations_seed_$(i).png")
end

In [ ]:
@show length(visit_orders)

In [ ]:
irs = [GPU.load_results(ic,:intrinsic_reward) for ic in best_per_algo_ics]

@show "here"
p = plot()
for (i,ir) in enumerate(irs)
    @show size(ir)
    l = mean(ir[:,1,:], dims=2)
    l = GPU.smooth(l,200)
    plot!(l,label=labels[i], palette=:tab10, ylim=(0,0.15))
end
title!("Intrinsic Reward")
xlabel!("Steps (per 100)")
ylabel!("Intrinsic Reward")
display(p)
    

In [ ]:
savefig("./plots/$(folder_name)/intrinsic_reward.png")

In [ ]:
num_gvfs = 4
ps = []
for i in 1:length(visit_orders)
    visitation = zeros(num_gvfs, num_gvfs)
    for run in 1:length(visit_orders[i])
        if length(visit_orders[i][run]) != 0
            first_visit = visit_orders[i][run][1]
#         @show first_visit
            goals = unique(visit_orders[i][run])
#         @show goals
            visitation[first_visit,goals] .+= 1
        end
    end
    @show maximum(visitation,dims=2)
    visitation = visitation ./ vec(maximum(visitation,dims=2))
    push!(ps, heatmap(visitation, title = labels[i], size = (1000,1300), c = :grayC))

end
plot(ps..., layout=(2,2))

In [ ]:
data_per_gvf = [GPU.get_stats(GPU.load_results(ic,data_key), per_gvf=true)[1] for ic in best_per_algo_ics]
std_per_gvf = [GPU.get_stats(GPU.load_results(ic,data_key), per_gvf=true)[2] for ic in best_per_algo_ics]

In [ ]:
GPU.get_stats(GPU.load_results(best_per_algo_ics[1],data_key), per_gvf=true)[2]

In [ ]:
data[1] .== data[3]

In [ ]:
inds_to_replicate

In [ ]:
ps = []
for ind in inds_to_replicate
    label_subset = labels[:,ind]
    data_subset = data_per_gvf[ind]
    std_subset = std_per_gvf[ind]
    @show size(std_subset)
    xticks=collect(step_increment:step_increment:num_samples*step_increment)
    p = plot(xticks,data_subset',label= ["Distractor" "Const" "Drifter" "Const"], ribbon = (std_subset/sqrt(100))', legend=:bottomleft, title=labels[ind], ylims=(0,1.25))
    push!(ps,p)
end

In [ ]:
plot(ps..., layout=(2,2))

In [ ]:
savefig("./plots/TabTMazeRoundRobin_per_gvf.png")

In [ ]:
# GPU.load_results(ic,:max_behaviour_q)
max_qs = [GPU.load_results(ic,:max_behaviour_q, return_type="array") for ic in best_per_algo_ics]

ps = []
for i in 1:length(best_per_algo_ics)
    p = plot()
    tmp = mean(cat([q for q in max_qs[i]]..., dims = 2), dims =2)
    @show minimum(tmp)
    plot!(tmp, size = (500,500), title=labels[i], label="")
    ylabel!(" Max Q value")
    xlabel!("steps")
    push!(ps, p)
end
plot(ps..., size = (1000,1000), layout = (1,2))

In [ ]:
savefig("./plots/$(folder_name)/max_q.png")

In [ ]:
max_qs = [GPU.load_results(ic,:max_behaviour_q, return_type="array") for ic in best_per_algo_ics]

ps = []
for i in 1:length(best_per_algo_ics)
    p = plot()
    tmp = mean(cat([q for q in max_qs[i]]..., dims = 2), dims =2)
    @show minimum(tmp)
    plot!(tmp, size = (500,500), title=labels[i], label="", ylim=(-0.1,0.1))
    ylabel!(" Max Q value")
    xlabel!("steps")
    push!(ps, p)
end
plot(ps..., size = (1000,1000), layout = (3,2))

In [ ]:
savefig("./plots/$(folder_name)/enhance_Enhance_ENHANCE.png")